In [10]:
using StatsPlots

In [11]:
TIME = 300
α1, α2, β, γ1, γ2, δ, ϵ = 0.5, 0.05, 0.7, 0.05, 0.8, 0.005, 0.3
C, I, G, W, P, Pf, Ph, T = zeros(TIME), zeros(TIME), zeros(TIME), zeros(TIME), zeros(TIME), zeros(TIME), zeros(TIME), zeros(TIME)
ΔH, ΔK, H, K = zeros(TIME), zeros(TIME), zeros(TIME), zeros(TIME)
HpY = zeros(TIME)
G0, K0, H0 = 1.0, 1.0, 1.0
G, K[1], H[1], C[1] = G0.*exp.(δ.*(0:(TIME-1))), K0, H0, (1-ϵ)/ϵ*G0+α2*H0
T[1] = ϵ*(C[1]+G0)
Ph[1] = (1 - β)*(C[1]+G0)
Pf[1] = (1-γ1)/(1-γ2)*(C[1]+G0)+(γ1-γ2)/(1-γ2)*K0
I[1] = Pf[1]
P[1] = Pf[1] + Ph[1]
W[1] = β*(C[1]+G0)
ΔH[1] = G0-T[1]
ΔK[1] = 1/(1 - γ1)*(I[1] - γ1*K0)

12.916666666666668

In [12]:
function run(ϵ1::Float64)
    for t=2:TIME
        if t < TIME*2÷3 + 1
            C[t] = (1 - ϵ)*(W[t-1] + α1*Ph[t-1]) + α2*H[t-1]
        else
            C[t] = (1 - ϵ1)*(W[t-1] + α1*Ph[t-1]) + α2*H[t-1]
        end
        I[t] = max(0, C[t] + G[t] + (γ1 - γ2)*K[t-1])
        W[t] = β*(C[t] + G[t])
        P[t] = C[t] + G[t] + I[t] - W[t]
        Pf[t] = I[t]
        Ph[t] = P[t] - Pf[t]
        if t < TIME*2÷3
            T[t] = ϵ*(W[t] + Ph[t])
        else
            T[t] = ϵ1*(W[t] + Ph[t])
        end
        ΔH[t] = G[t] - T[t]
        ΔK[t] = I[t] - γ1*K[t-1]
        H[t] = H[t-1] + ΔH[t]
        K[t] = K[t-1] + ΔK[t]

        HpY[t] = H[t]/(C[t] + I[t] + G[t])
    end
end
ϵ_min, step, ϵ_max = 0.1, 0.1, 0.5
s = (round(Int, TIME÷1.7)-TIME*2÷3):(TIME-TIME*2÷3)
function ϵs_plots(X, name)
    plot(s,zeros(length(s)), color="white", label=nothing, title="variable="*name*", ϵ=$ϵ")
    for ϵ1=ϵ_min:step:ϵ_max
        Δϵ = (10*ϵ1 - 10*ϵ)/10
        run(ϵ1)
        plot!(s, X[round(Int, TIME÷1.7):TIME], label="Δϵ=$Δϵ")
    end
    savefig("figs/"*name*".png")
end
ϵs_plots(C, "C")
ϵs_plots(T, "T")
ϵs_plots(I, "I")
ϵs_plots(W, "W")
ϵs_plots(P, "P")
ϵs_plots(Pf, "Pf")
ϵs_plots(Ph, "Ph")
ϵs_plots(ΔH, "ΔH")
ϵs_plots(ΔK, "ΔK")
ϵs_plots(H, "H")
ϵs_plots(K, "K")
ϵs_plots(HpY, "H÷Y")

"c:\\Users\\user\\OneDrive\\ドキュメント\\jupyter\\SFC_projects\\tax_rate_effects_in_SFC\\figs\\H÷Y.png"

In [15]:
#   バブルが始まるころからの日本をなんとなくイメージ
function run(G::Vector{Float64})
    for t=2:TIME
        C[t] = (1 - ϵ)*(W[t-1] + α1*Ph[t-1]) + α2*H[t-1]
        I[t] = max(0, C[t] + G[t] + (γ1 - γ2)*K[t-1])
        W[t] = β*(C[t] + G[t])
        P[t] = C[t] + G[t] + I[t] - W[t]
        Pf[t] = I[t]
        Ph[t] = P[t] - Pf[t]
        T[t] = ϵ*(W[t] + Ph[t])
        ΔH[t] = G[t] - T[t]
        ΔK[t] = I[t] - γ1*K[t-1]
        H[t] = H[t-1] + ΔH[t]
        K[t] = K[t-1] + ΔK[t]

        HpY[t] = H[t]/(C[t] + I[t] + G[t])
    end
end
G[1] = G0
δ = 0.02
for i=2:TIME*2÷3
    G[i] = exp(δ)*G[i-1]
end
for i=TIME*2÷3:TIME
    G[i] = G[i-1]
end
run(G)
s = (round(Int, TIME÷1.7)-TIME*2÷3):(TIME-TIME*2÷3)
function ϵs_plots(X, name)
    plot(s,zeros(length(s)), color="white", label=nothing, title="variable="*name)
    run(G)
    plot!(s, X[round(Int, TIME÷1.7):TIME], label=nothing)
    savefig("figs/_Gdif_"*name*".png")
end
ϵs_plots(C, "C")
ϵs_plots(T, "T")
ϵs_plots(I, "I")
ϵs_plots(W, "W")
ϵs_plots(P, "P")
ϵs_plots(Pf, "Pf")
ϵs_plots(Ph, "Ph")
ϵs_plots(ΔH, "ΔH")
ϵs_plots(ΔK, "ΔK")
ϵs_plots(H, "H")
ϵs_plots(K, "K")
ϵs_plots(HpY, "H÷Y")

"c:\\Users\\user\\OneDrive\\ドキュメント\\jupyter\\SFC_projects\\tax_rate_effects_in_SFC\\figs\\_Gdif_H÷Y.png"

In [23]:
#   政府支出を前期の税収の一定倍にする圧力を追加する
function endogenous_G_run(ϵ1, χ)
    for t=2:TIME
        if t < TIME*2÷3 + 1
            C[t] = (1 - ϵ)*(W[t-1] + α1*Ph[t-1]) + α2*H[t-1]
        else
            C[t] = (1 - ϵ1)*(W[t-1] + α1*Ph[t-1]) + α2*H[t-1]
        end
        G[t] = (exp(δ)*G[t-1] + (1+χ)*T[t-1])/2
        I[t] = max(0, C[t] + G[t] + (γ1 - γ2)*K[t-1])
        W[t] = β*(C[t] + G[t])
        P[t] = C[t] + G[t] + I[t] - W[t]
        Pf[t] = I[t]
        Ph[t] = P[t] - Pf[t]
        if t < TIME*2÷3
            T[t] = ϵ*(W[t] + Ph[t])
        else
            T[t] = ϵ1*(W[t] + Ph[t])
        end
        ΔH[t] = G[t] - T[t]
        ΔK[t] = I[t] - γ1*K[t-1]
        H[t] = H[t-1] + ΔH[t]
        K[t] = K[t-1] + ΔK[t]

        HpY[t] = H[t]/(C[t] + I[t] + G[t])
    end
end
χ, δ = 0.05, 0.005
ϵ_min, step, ϵ_max = 0.1, 0.1, 0.5
s = (round(Int, TIME÷1.7)-TIME*2÷3):(TIME-TIME*2÷3)
function ϵs_plots(X, name)
    plot(s,zeros(length(s)), color="white", label=nothing, title="variable="*name*", ϵ=$ϵ")
    for ϵ1=ϵ_min:step:ϵ_max
        Δϵ = (10*ϵ1 - 10*ϵ)/10
        endogenous_G_run(ϵ1, χ)
        plot!(s, X[round(Int, TIME÷1.7):TIME], label="Δϵ=$Δϵ")
    end
    savefig("figs/_endgeneousG_"*name*".png")
end
ϵs_plots(C, "C")
ϵs_plots(T, "T")
ϵs_plots(I, "I")
ϵs_plots(W, "W")
ϵs_plots(P, "P")
ϵs_plots(Pf, "Pf")
ϵs_plots(Ph, "Ph")
ϵs_plots(ΔH, "ΔH")
ϵs_plots(ΔK, "ΔK")
ϵs_plots(H, "H")
ϵs_plots(K, "K")
ϵs_plots(HpY, "H÷Y")
ϵs_plots(G, "G")

"c:\\Users\\user\\OneDrive\\ドキュメント\\jupyter\\SFC_projects\\tax_rate_effects_in_SFC\\figs\\_endgeneousG_G.png"

Gの水準の意思決定に、前期の税収の一定倍に近づくように圧力がかかっていると、税率を上げるほうが政府支出が増えやすくなり結果的に（名目で）経済成長する可能性を示した形

現実では増税したときに「一時的にGDP成長率が下がってから高成長を始める」という現象を見ないので、現実でよくある意思決定ではない可能性が高いと思う。税収を参照するのではなく、前年の予算を参照するのが普通という場合は、政府の支出を純粋に外生的に与えるモデルが優秀だと思う